# Setup

In [13]:
import matplotlib
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go

from ipywidgets import interact, widgets

from car_cost_calculator.car_costs import CarCosts
from car_cost_calculator.running_costs import RunningCosts
from car_cost_calculator.depreciation import TwoStageRate

In [14]:
np.set_printoptions(suppress=True)

In [15]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('seaborn-darkgrid')

# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [16]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

In [19]:
cost_order = ['depreciation', 'fuel', 'service', 'registration', 'insurance', 'tyres', 'roadside_assist', 'detailing']

In [20]:
@interact(
    use_matplotlib=widgets.Checkbox(value=False, description='Use matplotlib'),
    purchase=(0, 100000, 1000),
    fuel_economy=(0, 20, 0.1),
    km_per_year=(0, 100000, 1000),
    age=(0, 10, 1)
)
def explore(
    purchase=35000,
    age=0,
    fuel_economy=8.0,
    km_per_year=15000,
    use_matplotlib=False
):
    
    cc = CarCosts(
        initial_vehicle_value=purchase,
        initial_vehicle_age=age,
        depreciation_rate=TwoStageRate(
            stage_1_rate=0.15, stage_2_rate=0.1, breakpoint=3),
        years=10,
        km_per_year=km_per_year,
        litres_per_100km=fuel_economy,
        inflation=0.02,
        initial_fuel_price=1.50,
        initial_service_cost=350,
        service_interval_km=10000,
        service_interval_years=1,
        tyre_replacement_interval=30000,
        initial_cost_per_tyre=300)
    
    cc.yearly_costs.index += 1
    
    if use_matplotlib:
        cc.yearly_costs.plot.bar(stacked=True, figsize=(12,10))
    else:
        yc = cc.yearly_costs
        data = [go.Bar(x=cc.yearly_costs.index, y=cc.yearly_costs[cost], name=cost) for cost in cost_order]
        layout = go.Layout(barmode='stack', title='Yearly Costs')
        fig = go.Figure(layout=layout, data=data)
        py.iplot(fig, config=config)

interactive(children=(IntSlider(value=35000, description='purchase', max=100000, step=1000), IntSlider(value=0…